In [1]:
import json
import numpy as np
import obi_one as obi
import os
import pandas as pd
from conntility.connectivity import ConnectivityMatrix

In [2]:
input_root = "/Users/pokorny/Data/Circuits"
cmat_root = "/Users/pokorny/Data/ConnectivityMatrices"
output_root = "../../../obi-output/extracted_small_microcircuits_simplex"

In [3]:
# Load nbS1-O1 circuit with connectivity matrix
circuit = obi.Circuit(name="nbS1-O1",
                      path=os.path.join(input_root, "nbS1-O1", "circuit_config_postfix2.json"),
                      matrix_path=os.path.join(cmat_root, "nbS1-O1", "connectivity_matrix.h5"))
display(circuit.connectivity_matrix.matrix)

<COOrdinate sparse matrix of dtype 'int64'
	with 87457567 stored elements and shape (211712, 211712)>

In [4]:
# nbS1-O1 - Central neurons in connectivity
df_sel = pd.read_pickle("Top10CentralNeurons_Layer_Hex.pkl")  # (Contains neuron IDs relative to nbS1-HEXn)
# df_opt = pd.read_pickle("Top10CentralNeurons_Layer_Hex_OptIDX.pkl")  # Don't used here

# Load HEXi connectivity matrix
base_name = "nbS1"
hex_sel = "hex0"
layers = range(1, 7)
cmat = ConnectivityMatrix.from_h5(os.path.join(cmat_root, f"{base_name}-{hex_sel.upper()}", "connectivity_matrix.h5"))
ntab = cmat.vertices.set_index("node_ids")

# Load ID mapping w.r.t. parent circuit nbS1-O1
id_map_file = os.path.join(input_root, f"{base_name}-{hex_sel.upper()}", "id_mapping.json")
with open(id_map_file, "r") as f:
    id_map = json.load(f)
assert np.array_equal(id_map["S1nonbarrel_neurons"]["new_id"], ntab.index.values)

nid_selection = []  # Selected neuron IDs for nbS1-O1 circuit
for _lay in layers:
    # opt_idx = df_opt.loc[(hex_sel, _lay)]["pos"]
    opt_idx = 0  # Always take first one from list, which is the most central neuron
    node_id = df_sel[("hex0", _lay)].iloc[opt_idx].tolist()
    nid_selection.append(id_map["S1nonbarrel_neurons"]["old_id"][node_id])

# Check selection
cmat = circuit.connectivity_matrix
sel_tab = cmat.vertices.loc[nid_selection]
assert np.array_equal(sel_tab["layer"].astype(int), layers)
display(sel_tab)

,node_ids,etype,layer,mtype,synapse_class,x,y,z
426,426,bNAC,1,L1_DAC,INH,4471.583630,-1465.951445,-1787.636653
151361,151361,cNAC,2,L23_CHC,INH,4417.154568,-1455.019186,-1899.847804
25652,25652,cACint,3,L23_CHC,INH,4426.676668,-1365.720303,-1999.555547
128851,128851,cADpyr,4,L4_TPC,EXC,4190.372099,-1349.730546,-2291.566707
191094,191094,cADpyr,5,L5_TPC:B,EXC,3838.789927,-1299.677721,-2664.707385
79088,79088,cADpyr,6,L6_IPC,EXC,3592.676277,-1025.078101,-2898.148159


In [5]:
# Set up simplex neuron sets
N = 10  # Max. number of neurons
# dim = 10  # Top dim (may be actually 4, 5 or 6)
dim = 2  # Triplets
# simplex_pos = "source"
simplex_pos = "target"
simplex_neuron_sets = obi.SimplexNeuronSet(
    central_neuron_id=nid_selection,
    dim=dim,
    central_neuron_simplex_position=simplex_pos,
    subsample=True,
    seed=0,
    n_count_max=N,
    simplex_type="directed",
    property_filter=obi.scientific.circuit.neuron_sets.NeuronPropertyFilter(),
)

virtual_sources_to_ignore = ("external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical", )
output_path = os.path.join(output_root, f"dim{dim}-{simplex_pos}")

In [6]:
circuit_extractions_form = obi.CircuitExtractions(
                    initialize=obi.CircuitExtractions.Initialize(
                        circuit=circuit,
                        run_validation=False,
                        do_virtual=True,
                        create_external=True,
                        virtual_sources_to_ignore=virtual_sources_to_ignore,
                    ),
                    neuron_set=simplex_neuron_sets,
)

coupled_scan = obi.CoupledScan(form=circuit_extractions_form, output_root=output_path, coordinate_directory_option="ZERO_INDEX")

In [7]:
# Display the intermediary data
coupled_scan.multiple_value_parameters(display=True)
coupled_scan.coordinate_parameters(display=True)
coupled_scan.coordinate_instances(display=True)

[2025-06-19 16:54:43,232] INFO: 
MULTIPLE VALUE PARAMETERS
[2025-06-19 16:54:43,234] INFO: neuron_set.central_neuron_id: [426, 151361, 25652, 128851, 191094, 79088]
[2025-06-19 16:54:43,235] INFO: 
COORDINATE PARAMETERS
[2025-06-19 16:54:43,236] INFO: neuron_set.central_neuron_id: 426
[2025-06-19 16:54:43,237] INFO: neuron_set.central_neuron_id: 151361
[2025-06-19 16:54:43,237] INFO: neuron_set.central_neuron_id: 25652
[2025-06-19 16:54:43,238] INFO: neuron_set.central_neuron_id: 128851
[2025-06-19 16:54:43,238] INFO: neuron_set.central_neuron_id: 191094
[2025-06-19 16:54:43,239] INFO: neuron_set.central_neuron_id: 79088
[2025-06-19 16:54:43,241] INFO: 
COORDINATE INSTANCES
[2025-06-19 16:54:43,241] INFO: CircuitExtraction(idx=0, scan_output_root=PosixPath('.'), coordinate_output_root=PosixPath('.'), single_coordinate_scan_params=SingleCoordinateScanParams(type='SingleCoordinateScanParams', scan_params=[SingleValueScanParam(type='SingleValueScanParam', location_list=['neuron_set', 'cen

[CircuitExtraction(idx=0, scan_output_root=PosixPath('.'), coordinate_output_root=PosixPath('.'), single_coordinate_scan_params=SingleCoordinateScanParams(type='SingleCoordinateScanParams', scan_params=[SingleValueScanParam(type='SingleValueScanParam', location_list=['neuron_set', 'central_neuron_id'], value=426)], nested_coordinate_subpath_str=PosixPath('.')), type='CircuitExtraction', initialize=Initialize(type='CircuitExtractions.Initialize', circuit=Circuit(type='Circuit', name='nbS1-O1', path='/Users/pokorny/Data/Circuits/nbS1-O1/circuit_config_postfix2.json', matrix_path='/Users/pokorny/Data/ConnectivityMatrices/nbS1-O1/connectivity_matrix.h5'), run_validation=False, do_virtual=True, create_external=True, virtual_sources_to_ignore=('external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical',)), neuron_set=SimplexNeuronSet(type='SimplexNeuronSet', random_sample=None, random_seed=0, node_population=None, property_filter=NoFilter, node_sets=(), central_neuron_id=426, dim=2, centra

In [8]:
coupled_scan.execute(processing_method='run')

[2025-06-19 16:54:44,791] INFO: None
[2025-06-19 16:56:00,582] INFO: COMPUTE list of simplices by dimension
Extracting subcircuit from 'nbS1-O1'
Copying morphologies for population 'S1nonbarrel_neurons' (10)


Copying containerized .h5 morphologies: 100%|██████████████████████████████████████████████████| 10/10 [00:00<00:00, 677.05it/s]

Copied 10 morphologies into container (0 already existed)
Copying 5 biophysical neuron models (.hoc) for population 'S1nonbarrel_neurons' (10)
Copying mod files
Extraction DONE


[2025-06-19 17:00:41,664] INFO: COMPUTE list of simplices by dimension
Extracting subcircuit from 'nbS1-O1'
Copying morphologies for population 'S1nonbarrel_neurons' (10)


Copying containerized .h5 morphologies: 100%|██████████████████████████████████████████████████| 10/10 [00:00<00:00, 862.95it/s]

...

Copied 10 morphologies into container (0 already existed)
Copying 5 biophysical neuron models (.hoc) for population 'S1nonbarrel_neurons' (10)
Copying mod files
Extraction DONE


[2025-06-19 17:17:40,660] INFO: COMPUTE list of simplices by dimension
Extracting subcircuit from 'nbS1-O1'
Copying morphologies for population 'S1nonbarrel_neurons' (10)


Copying containerized .h5 morphologies: 100%|██████████████████████████████████████████████████| 10/10 [00:00<00:00, 921.66it/s]

Copied 10 morphologies into container (0 already existed)
Copying 2 biophysical neuron models (.hoc) for population 'S1nonbarrel_neurons' (10)
Copying mod files
Extraction DONE
[2025-06-19 17:20:47,952] INFO: create_bbp_workflow_campaign_config() not yet complete.


{0: None, 1: None, 2: None, 3: None, 4: None, 5: None}

In [9]:
# Check circuits
from bluepysnap import Circuit

for _idx, _inst in enumerate(coupled_scan.coordinate_instances()):
    cfg = coupled_scan.output_root / str(_idx) / "circuit_config.json"
    c = Circuit(cfg)
    nodes = c.nodes['S1nonbarrel_neurons']
    mtypes = np.unique(nodes.get(properties=['mtype']), return_counts=True)
    mt_hist = [f"{_c}x{_mt}" for _mt, _c in zip(*mtypes)]
    print(f"L{layers[_idx]}: {nodes.size} neurons, {c.edges["S1nonbarrel_neurons__S1nonbarrel_neurons__chemical"].size} synapses, m-types: {', '.join(mt_hist)}")


L1: 10 neurons, 125 synapses, m-types: 1xL1_DAC, 3xL2_TPC:B, 1xL3_TPC:A, 2xL3_TPC:C, 1xL4_TPC, 2xL5_TPC:A
L2: 10 neurons, 64 synapses, m-types: 1xL23_CHC, 1xL3_TPC:A, 3xL3_TPC:C, 4xL5_TPC:A, 1xL5_TPC:B
L3: 9 neurons, 83 synapses, m-types: 1xL23_CHC, 1xL2_TPC:B, 3xL4_TPC, 1xL4_UPC, 2xL5_MC, 1xL5_TPC:C
L4: 10 neurons, 107 synapses, m-types: 1xL23_LBC, 4xL2_TPC:B, 2xL4_SSC, 1xL4_TPC, 1xL5_MC, 1xL5_TPC:B
L5: 10 neurons, 154 synapses, m-types: 1xL4_TPC, 1xL4_UPC, 2xL5_LBC, 1xL5_MC, 1xL5_TPC:B, 2xL6_HPC, 2xL6_IPC
L6: 10 neurons, 100 synapses, m-types: 3xL5_TPC:A, 2xL5_TPC:B, 2xL6_HPC, 2xL6_IPC, 1xL6_TPC:C
